# [![](https://img.shields.io/badge/Kohya%20Setting-Wiki-Blue)](https://github.com/kohya-ss/sd-scripts/tree/sd3?tab=readme-ov-file#flux1-fine-tuning) [![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/UCLlbxQD0uk) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-1.0-0075ff)](https://trainlora.vn) [![](https://img.shields.io/badge/All%20Tools-sdvn.me-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/badge/Khoá%20học-SD%20Flux-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---
`✨ Fix resume train`

#💽 Install

In [ ]:
# @title ###💽 Download + Install
import os
from google.colab import drive
drive.mount('/content/drive')

data_dir = "/content/drive/MyDrive/SD-Data"
repo_dir = "/content/SDVN-training-colab-flux"

TrainType = "Flux" # @param ["Flux", "SDXL", "SD15"]

brand_d = {
    "Flux": "sd3",
    "SDXL": "main",
    "SD15": "main"
}

model_folder = "/content/model"

#Install
!git clone -b dev https://github.com/StableDiffusionVN/SDVN-training-colab-flux
!pip install -q -r /content/SDVN-training-colab-flux/requirements.txt

if os.path.exists('/content/sd-scripts'):
  %cd /content/sd-scripts
  !git checkout {brand_d[TrainType]}
else:
  %cd /content
  !git clone -b {brand_d[TrainType]} https://github.com/kohya-ss/sd-scripts
  !sed -i 's/^bitsandbytes==0.44.0/bitsandbytes/' /content/sd-scripts/requirements.txt
if TrainType != "Flux":
  !sed -i 's/^# for Image utils/numpy<2.0/' /content/sd-scripts/requirements.txt

%cd /content/sd-scripts
!pip install -q -r /content/sd-scripts/requirements.txt
%cd /content
%run {repo_dir}/TrainScript.ipynb

#📂 II. Xử lý dữ liệu

In [ ]:
#@title ✨ 2.1 Caption
#@markdown [![](https://img.shields.io/badge/Caption-Tools-blue)](https://colab.research.google.com/github/StableDiffusionVN/SDVN-training-colab-flux/blob/main/SDVN_Caption_Tools.ipynb)

TrainFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
No_gen_caption = False #@param {type:"boolean"}
# @markdown 🟡 `OCR Prompt`

#@markdown 💡 ` Florence yêu cầu khởi động lại phiên (Ctr + M + .) và chạy lại từ đầu`
Caption = 'None' # @param ['None', 'WD14','Florence', 'APIGemini | 2.0 Flash', 'APIGemini | 2.0 Flash Lite', 'APIGemini | 2.5 Flash Preview', ' APIGemini | 2.5 Pro Preview', 'APIOpenAI | GPT 4-o mini', 'APIOpenAI | GPT o4-mini']
Caption_Length = "Medium" # @param ["Short","Medium","Long"]
# @markdown 🟡 `API Caption`
APIkey = "" # @param {type:'string'}
API_Prompt = "" # @param {type:'string'}
# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}

if No_gen_caption == True:
  Caption = 'None'
  Custom_Caption = ''
  AddFolderName = False

Cap_prompt = {
    'Short':['<CAPTION>',0.85, 30],
    'Medium':['<DETAILED_CAPTION>',0.5, 60],
    'Long':['<MORE_DETAILED_CAPTION>',0.35, 100]
}
extension =".txt"

if DataClean == True :
  %cd /content
  clean_directory(TrainFolder)
if Caption != "None":
  caption_dir(TrainFolder,Cap_prompt[Caption_Length])
if AddFolderName:
  add_forder_name(TrainFolder)
if Custom_Caption != "":
  process_dir(TrainFolder, Custom_Caption, Append, Remove_Caption)


#⏳ III. Training Lora

In [ ]:
#@title ###⚡️ Cấu hình train Lora
%run {repo_dir}/Config.ipynb

output_name = "lora_name" # @param {"type":"string"}
output_dir = "/content/drive/MyDrive/SD-Data/Lora" # @param {"type":"string"}

#@markdown `💡 Nhận đường dẫn thư mục, đường dẫn tải về, địa chỉ civitAI, huggingface, danh sách trong list có sẵn. Mặc định Flux dùng model gốc`
model_train = "" #@param ["","RealisticVision51", "SDXL-Base", "STOIQO-NewRealityXLPro", "RealVisXLv5", "CharacterArtXL", "RealMythXL_v7", "XXMix_9realisticSDXL", "LEOSAM-HelloWorldXL", "AdamXL-v3", "JuggernautXL-v9", "Illustrious-XL", "PonyDiffusionV6XL"] {allow-input: true}
image_dir = TrainFolder
#@markdown `💡 Nhập nhiều size để train nhiều kích thước (chỉ dùng cho flux), vd: 768,1024,1280; chỉ định batch riêng cho từng size, vd: 768-6,1024-4,1280-2`
resolution = "1024" # @param {"type":"string"}
#@markdown `💡 Điều chỉnh theo mức độ tiêu thụ GPU(phụ thuộc GPU, resolution, optimizer_type)`
batch_size = 2 # @param {"type":"number"}
flip = False # @param {"type":"boolean"}
color_aug = False # @param {"type":"boolean"}
shuffle_caption = False # @param {"type":"boolean"}
#@markdown `💡 Có thể điều chỉnh repeat cho từng thư mục bằng cách đặt tên thư mục theo cú pháp {repeat}_{folder_name}`
num_repeats = 24 # @param {"type":"number"}
epochs = 4 # @param {"type":"number"}
save_every_n_epochs = 1 # @param {"type":"number"}
save_every_n_steps = 0 # @param {"type":"number"}
lr = 1e-4 # @param {"type":"number"}
lr_scheduler = "constant" # @param ["constant","cosine","linear","cosine_with_restarts","adafactor","polynomial","constant_with_warmup"]
dim = 32 # @param {"type":"number"}
alpha = 16 # @param {"type":"number"}
optimizer_type = 'adamw8bit' # @param ["adafactor","adamw8bit"]
train_textencode = True # @param {"type":"boolean"}

#@markdown `*️⃣ Tuỳ chọn mở rộng`
Extra = True # @param {"type":"boolean"}
author = '' # @param {"type":"string"}
#@markdown `💡 Gen ảnh sample hiện không khả dụng với SD15`
sample_prompt = '' # @param {"type":"string"}
sample_size = '768,1024' # @param {"type":"string"}
sample_steps = 100 # @param {"type":"number"}
save_state = False # @param {"type":"boolean"}
resume = '' # @param {"type":"string"}
wandbapi = '' # @param {"type":"string"}
#@markdown `☕️ Training`
runcode = False # @param {"type":"boolean"}
output_dir = f"{output_dir}/{output_name}"
dataset = f'/content/dataset_{output_name}.toml'
AutoDisconect = False # @param {"type":"boolean"}

#Config
if TrainType == 'SD15':
  sample_steps =0

model_train = flux_path if TrainType == 'Flux' else download_lib(model_train)
data_config = data_config(flip, color_aug, shuffle_caption)
config = config(TrainType, model_train, flux_path, clip_l_path, t5xxl_fp16_path, vae_path, dim, alpha, train_textencode, optimizer_type, lr, lr_scheduler, epochs, save_every_n_epochs, save_every_n_steps, dataset, output_dir, output_name)
extra = extra(TrainType, output_name, author,save_state, resume, wandbapi, sample_steps, runcode)
prompt(sample_prompt, sample_size, TrainFolder)
dataset_file(resolution, batch_size, image_dir, num_repeats, data_config, dataset)
arg = f'accelerate launch {dic2arg(op)} {lora_train_py[TrainType]} {dic2arg(config)} {dic2arg(extra) if Extra else ""}'
check_folder_train(image_dir)

if resume != "":
  !pip install -q -U accelerate

print('Chạy lệnh sau trong Terminal( Cửa sổ dòng lệnh) hoặc bật chế độ runcode (Sẽ không hiển thị thông báo):')
print(f"cd /content/sd-scripts;{arg}")
if runcode:
  %cd /content/sd-scripts
  !{arg}
if AutoDisconect:
  from time import sleep
  sleep(3*60)
  from google.colab import runtime
  runtime.unassign()

#⏳ IV. Training Checkpoint

In [ ]:
#@title ###⚡️ Cấu hình train Checkpoint
%run {repo_dir}/Config.ipynb
output_name = "checkpoint_name" # @param {"type":"string"}
output_dir = "/content/drive/MyDrive/SD-Data/Model" # @param {"type":"string"}
#@markdown `💡 Nhận đường dẫn thư mục, đường dẫn tải về, địa chỉ civitAI, huggingface, danh sách trong list có sẵn. Mặc định Flux dùng model gốc`
model_train = "" #@param ["","RealisticVision51", "SDXL-Base", "STOIQO-NewRealityXLPro", "RealVisXLv5", "CharacterArtXL", "RealMythXL_v7", "XXMix_9realisticSDXL", "LEOSAM-HelloWorldXL", "AdamXL-v3", "JuggernautXL-v9", "Illustrious-XL", "PonyDiffusionV6XL"] {allow-input: true}
image_dir = TrainFolder
#@markdown `💡 Nhập nhiều size để train nhiều kích thước (chỉ dùng cho flux), vd: 768,1024,1280; chỉ định batch riêng cho từng size, vd: 768-6,1024-4,1280-2`
resolution = "1024" # @param {"type":"string"}
#@markdown `💡 Điều chỉnh theo mức độ tiêu thụ GPU(phụ thuộc GPU, resolution, optimizer_type)`
batch_size = 1 # @param {"type":"number"}
flip = False # @param {"type":"boolean"}
color_aug = False # @param {"type":"boolean"}
shuffle_caption = False # @param {"type":"boolean"}
#@markdown `💡 Có thể điều chỉnh repeat cho từng thư mục bằng cách đặt tên thư mục theo cú pháp {repeat}_{folder_name}`
num_repeats = 10 # @param {"type":"number"}
epochs = 4 # @param {"type":"number"}
save_every_n_epochs = 1 # @param {"type":"number"}
save_every_n_steps = 0 # @param {"type":"number"}
lr = 1e-6 # @param {"type":"number"}
lr_scheduler = "constant" # @param ["constant","cosine","linear","cosine_with_restarts","adafactor","polynomial","constant_with_warmup"]
#@markdown `💡 Chọn Adafactor để giảm tải GPU Flux/SDXL (Sẽ vô hiệu Lr Scheduler)`
optimizer_type = 'adamw8bit' # @param ["adafactor","adamw8bit"]

#@markdown `*️⃣ Tuỳ chọn mở rộng`
Extra = True # @param {"type":"boolean"}
author = '' # @param {"type":"string"}
#@markdown `💡 Gen ảnh sample hiện không khả dụng với SD15`
sample_prompt = '' # @param {"type":"string"}
sample_size = '768,1024' # @param {"type":"string"}
sample_steps = 100 # @param {"type":"number"}
save_state = False # @param {"type":"boolean"}
resume = '' # @param {"type":"string"}
wandbapi = '' # @param {"type":"string"}
#@markdown `☕️ Training`
runcode = False # @param {"type":"boolean"}
output_dir = f"{output_dir}/{output_name}"
dataset = f'/content/dataset_{output_name}.toml'
AutoDisconect = False # @param {"type":"boolean"}

def config_cp(TrainType, model_train, flux_path, clip_l_path, t5xxl_fp16_path, vae_path, optimizer_type, lr, lr_scheduler, epochs, save_every_n_epochs, save_every_n_steps, dataset, output_dir, output_name):
  return {
    '--pretrained_model_name_or_path': flux_path if TrainType == 'Flux' else model_train,
    '--clip_l': clip_l_path if TrainType == 'Flux' else False,
    '--t5xxl': t5xxl_fp16_path if TrainType == 'Flux' else False,
    '--ae': vae_path if TrainType == 'Flux' else False,
    '--cache_latents_to_disk': True,
    '--save_model_as': 'safetensors',
    '--sdpa': True,
    '--persistent_data_loader_workers': True,
    '--max_data_loader_n_workers': 2,
    '--seed': 42,
    '--gradient_checkpointing': True,
    '--mixed_precision': 'bf16',
    '--save_precision': 'bf16',
    '--optimizer_type': optimizer_type,
    '--optimizer_args': '"relative_step=False" "scale_parameter=False" "warmup_init=False"' if optimizer_type == 'adafactor' else False,
    '--learning_rate': lr,
    '--lr_scheduler': 'constant_with_warmup' if optimizer_type == 'adafactor' else lr_scheduler,
    '--unet_lr': False,
    '--text_encoder_lr': False,
    '--cache_text_encoder_outputs': True if TrainType == 'Flux' else False,
    '--cache_text_encoder_outputs_to_disk': True if TrainType == 'Flux' else False,
    '--fp8_base': True if TrainType == 'Flux' else False,
    '--highvram': True,
    '--max_train_epochs': epochs,
    '--save_every_n_epochs': False if save_every_n_epochs <= 0 else save_every_n_epochs,
    '--save_every_n_steps': False if save_every_n_steps <= 0 else save_every_n_steps,
    '--dataset_config': f'"{dataset}"',
    '--output_dir': f'"{output_dir}"',
    '--output_name': f'"{output_name}"',
    '--timestep_sampling': 'shift' if TrainType == 'Flux' else False,
    '--discrete_flow_shift': 3.1582 if TrainType == 'Flux' else False,
    '--model_prediction_type': 'raw' if TrainType == 'Flux' else False,
    '--guidance_scale': 1.0 if TrainType == 'Flux' else False,
    '--max_bucket_reso': 2048,
    '--min_bucket_reso': 256 if TrainType == 'SD15' else 512,
  }
#Config
if TrainType == 'SD15':
  sample_steps =0

model_train = flux_path if TrainType == 'Flux' else download_lib(model_train)

data_config = data_config(flip, color_aug, shuffle_caption)
config = config_cp(TrainType, model_train, flux_path, clip_l_path, t5xxl_fp16_path, vae_path, optimizer_type, lr, lr_scheduler, epochs, save_every_n_epochs, save_every_n_steps, dataset, output_dir, output_name)
extra = extra_cp(TrainType, output_name, author,save_state, resume, wandbapi, sample_steps, runcode)
prompt(sample_prompt, sample_size, TrainFolder)
dataset_file(resolution, batch_size, image_dir, num_repeats, data_config, dataset)
arg = f'accelerate launch {dic2arg(op)} {db_train_py[TrainType]} {dic2arg(config)} {dic2arg(extra) if Extra else ""}'
check_folder_train(image_dir)

if resume != "":
  !pip install -q -U accelerate

print('Chạy lệnh sau trong Terminal( Cửa sổ dòng lệnh) hoặc bật chế độ runcode (Sẽ không hiển thị thông báo):')
print(f"cd /content/sd-scripts;{arg}")
if runcode:
  %cd /content/sd-scripts
  !{arg}
if AutoDisconect:
  from time import sleep
  sleep(3*60)
  from google.colab import runtime
  runtime.unassign()


#👨🏻‍💻 V.Tools and Dev

In [ ]:
# @title ⏳ RunSleep
#@markdown `Tác vụ vô dụng để tránh bị ngắt trong lúc train bằng terminal`
from time import sleep
Min = 200 # @param {"type":"number"}
sleep(Min*60)